<a href="https://colab.research.google.com/github/SalarNouri/Advanced-Deep-Learning/blob/main/pac_image/compute_pacs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Mount drive files
from google.colab import drive

drive.mount('/content/drive/')

In [ ]:
#@title Install tensorpac library

!pip install tensorpac

In [ ]:
#@title Define Libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import timeit
import pyarrow as pa
import pyarrow.parquet as pq

from random import sample
from scipy.io import loadmat
from tensorpac import Pac
from tensorpac.signals import pac_signals_tort
from IPython.display import clear_output

data = loadmat(f"/content/drive/MyDrive/Projects/UBC_PI/data/PT_TimeLocked_1000Back.MAT")


In [ ]:
#@title Parameter Initializing
channels = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11,
            12, 13, 14, 15, 16, 17, 18, 19, 20,
            21, 22, 23, 24, 25, 26, 27]

all_subjects = {"healthy": [1, 2, 3, 4, 5, 6, 7, 11, 12, 13, 15, 16, 17, 18, 19, 20, 21, 22],
                "parkinson_disease": [1, 2, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15, 16, 17, 18, 19, 20]
                }

peak_time = 12
reaction_time = 13
sampling_rate = 1000
trials = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

# keys = ['dataHCTask2', 'dataPD1Task2', 'dataPD2Task2']
tasks = ['task_gvssham.mat', 'task_gvsstim7.mat', 'task_gvsstim8.mat']
stimuli = ['sham', 'stim7', 'stim8']
# healthy_cases = ["hcoffmed", "hcoffmed", "hcoffmed_gvs8"]
# pd_off_cases = ["pdoffmed", "pdoffmed", "pdoffmed_gvs8"]
pd_on_cases = ["pdonmed", "pdonmed", "pdonmed_gvs8"]

keys = ['dataPD2Task2'] 
# tasks = ["task_gvsstim8.mat"]
# healthy_cases = ["hcoffmed_gvs8"]
# pd_on_cases = ["pdonmed_gvs8"]
# pd_off_cases = ["pdoffmed_gvs8"]
stim = 2

In [ ]:
#@title Obtain PAC function

def obtain_pac(chosed_signal):
    pac = Pac(idpac=(5, 2, 0), f_pha=(0, 30.1, 1, 1), f_amp=(0, 60.1, 1, 1), dcomplex='wavelet')
    xpac = pac.filterfit(sampling_rate, chosed_signal)
    return xpac.mean(-1)

In [ ]:
#@title Compute PAC values
data_stats = pd.DataFrame([])
run_id = 0

for key in keys:
    print(key)
    data_task = data[key]
    if key == 'dataHCTask2':
        subjects = all_subjects['healthy']
    else:
        subjects = all_subjects['parkinson_disease']

    print(stimuli[stim])
    task = loadmat(f"/content/drive/MyDrive/Projects/UBC_PI/data/EEG_Hackathon/{tasks[stim]}")
    
    # medication = healthy_cases[stim]
    # medication = pd_off_cases[stim]
    medication = pd_on_cases[stim]

    gvs_data = task[medication]
        
    for subject in subjects:
        # print(subject)
        eeg_data = data_task[stim, subject-1]
        # print(eeg_data.shape)
        for channel in channels:
            # print(channel)
            for trial in trials:
                behavior_signal = gvs_data.item(subject-1)
                p_time = behavior_signal[trial - 1, peak_time -1 ]
                r_time = behavior_signal[trial -1 , reaction_time -1]
                # print(trial)
                signal = eeg_data[channel-1, :, trial-1]
                start = timeit.default_timer()
                pac_values = obtain_pac(signal).reshape(1, 60*30)
                stop = timeit.default_timer()
                run_id = run_id + 1
                print('Run id: ', run_id)
                print('Time: ', stop - start)
                stats = {
                        "task": key,
                        "stimulus": stimuli[stim],
                        "medication": medication,
                        "channel": channel,
                        "subject": subject,
                        "trial": trial,
                        "peak_time": p_time,
                        "reaction_time": r_time,
                        "pac_values": pac_values[0]
                        }
                data_stats = data_stats.append(stats, ignore_index=True)
            clear_output(wait=True)

table = pa.Table.from_pandas(data_stats)
pq.write_table(table, 
               '/content/drive/MyDrive/Projects/UBC_PI/generated_data/pacs/data_9.parquet',
               compression='BROTLI')

# data_stats.to_csv(f"/content/drive/MyDrive/Projects/UBC_PI/generated_data/Statistics/temp.csv", 
#                     index=False)

In [ ]:
#@title Compute PAC values
data_stats = pd.DataFrame([])
run_id = 0
for key in keys:
    # print(key)
    if key == 'dataHCTask2':
        subjects = all_subjects['healthy']
        
    else:
        subjects = all_subjects['parkinson_disease']

    data_task = data[key]
    
    for stim in range(0, len(stimuli)):
        # print(stimuli[stim])
        task = loadmat(f"/content/drive/MyDrive/Projects/UBC_PI/data/EEG_Hackathon/{tasks[stim]}")
        
        # if stimuli[stim] == 'stim8':
        #     medication = healthy_cases[1]
        # else:
        #     medication = healthy_cases[0]

        # if stimuli[stim] == 'stim8':
        #     medication = pd_off_cases[1]
        # else:
        #     medication = pd_off_cases[0]
        # gvs_data = task[medication]

        if stimuli[stim] == 'stim8':
            medication = pd_on_cases[1]
        else:
            medication = pd_on_cases[0]
        gvs_data = task[medication]
            
        for subject in subjects:
            # print(subject)
            eeg_data = data_task[stim-1, subject-1]
            # print(eeg_data.shape)
            for channel in channels:
                # print(channel)
                for trial in trials:
                    behavior_signal = gvs_data.item(subject-1)
                    p_time = behavior_signal[trial - 1, peak_time -1 ]
                    r_time = behavior_signal[trial -1 , reaction_time -1]
                    # print(trial)
                    signal = eeg_data[channel-1, :, trial-1]
                    start = timeit.default_timer()
                    pac_vlaues = obtain_pac(signal).reshape(1, 60*30)
                    stop = timeit.default_timer()
                    run_id = run_id + 1
                    print('Run id: ', run_id)
                    print('Time: ', stop - start)
                    stats = {
                            "task": key,
                            "stimulus": stimuli[stim],
                            "medication": medication,
                            "channel": channel,
                            "subject": subject,
                            "trial": trial,
                            "peak_time": p_time,
                            "reaction_time": r_time,
                            "pac_values": pac_vlaues
                            }
                    data_stats = data_stats.append(stats, ignore_index=True)
                clear_output(wait=True)

data_stats.to_csv(f"/content/drive/MyDrive/Projects/UBC_PI/generated_data/Statistics/data_stats.csv", 
                    index=False)

In [ ]:
#@title Produce images temp

# data_stats = pd.DataFrame([])
data_stats = pd.read_csv("/content/drive/MyDrive/Projects/UBC_PI/generated_data/Statistics/data_stats_3_2.csv")
# img_idx = 0
img_idx = len(data_stats)

for (key_data, val_data), (key_task, val_task) in zip(data_names.items(), task_names.items()):
    print(key_data, val_data)
    print(key_task, val_task)
    data = loadmat(f"/content/drive/MyDrive/Projects/UBC_PI/data/EEG_Hackathon/{key_data}")
    task = loadmat(f"/content/drive/MyDrive/Projects/UBC_PI/data/EEG_Hackathon/{key_task}")

    for item, case in zip(val_data, val_task):
        eeg_data = data[item]
        task_data = task[case]

        if case in healthy_cases:
            subject = subjects["healthy"]
        else:
            subject = subjects["parkinson_disease"]
        
        for subject_idx in subject:
            signal = eeg_data.item(subject_idx - 1)
            behavior = task_data.item(subject_idx - 1)

            for trial in trials:
                for channel in channels:
                    stats = []
                    print(f"channel, trial and subject are {channel}, {trial} and {subject_idx}")
                    chosed_signal = signal[channel - 1, :, trial - 1]
                    p_time = behavior[trial - 1, peak_time -1 ]
                    r_time = behavior[trial -1 , reaction_time -1]
                    stats = {
                            "data_name": key_data,
                            "task_name": key_task,
                            "eeg_data": item,
                            "task_data": case,
                            "channel": channel,
                            "subject_id": subject_idx,
                            "trial": trial,
                            "peak_time": p_time,
                            "reaction_time": r_time
                            }

                    data_stats = data_stats.append(stats, ignore_index=True)
                    pac = obtain_pacimg(chosed_signal)
                    pac.savefig(f"{images_dir}/pac{img_idx}.jpg")
                    img_idx = img_idx + 1
                    clear_output(wait=True)
                    pac.show()
                    print(img_idx)

data_stats.to_csv(f"/content/drive/MyDrive/Projects/UBC_PI/generated_data/Statistics/data_stats_3_3.csv")